### 这个架构应该就是最终要实现的，到时候改一下 prompt 就行
### 另外要实现一些工具，就像search一样。

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import hub

In [2]:
prompt = hub.pull("hwchase17/react-chat-json")
chat_model = ChatOpenAI(temperature=0, model='gpt-4')

KeyboardInterrupt: 

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.utilities.serpapi import SerpAPIWrapper
from langchain.tools import Tool
from langchain.tools.render import render_text_description

search = SerpAPIWrapper(serpapi_api_key = "bb32d37fbdca24717613e25dbda90e9b6e34a663c863c94d4c5c4fcadb089f55")
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

chat_model_with_stop = chat_model.bind(stop=["\nObservation"])

from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_messages

# We need some extra steering, or the chat model forgets how to respond sometimes
TEMPLATE_TOOL_RESPONSE = """TOOL RESPONSE: 
---------------------
{observation}

USER'S INPUT
--------------------

Okay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else - even if you just want to respond to the user. Do NOT respond with anything except a JSON snippet no matter what!"""

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_messages(x['intermediate_steps'], template_tool_response=TEMPLATE_TOOL_RESPONSE),
    "chat_history": lambda x: x["chat_history"],
} | prompt | chat_model_with_stop | JSONAgentOutputParser()

from langchain.agents import AgentExecutor

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


In [ ]:
agent_executor.invoke({"input": "hi, i am bob"})['output']

In [ ]:
agent_executor.invoke({"input": "今天天气怎么样"})['output']


In [ ]:
agent_executor.invoke({"input": "what are some movies showing 9/21/2023?"})['output']


In [ ]:
prompt